In [8]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

<font size="5" color="red">ch15 데이터베이스 연동</font>
# 1절. sqlite 데이터 베이스 연동
- SQlite 데이터베이스는 별도의 dbms 없이 sql 을 이요하여 db 엑세스 가능한 디스크 기반 db
- SQLite는 프로토 타입 용도
- 프로젝트 단계 : 분석 - 설계 - 구현 - 테스트 - 고객에게 배포 - 유지보수
## 1.1 sqlite 패키지 load

In [5]:
import sqlite3
sqlite3.sqlite_version

'3.40.1'

In [7]:
import pandas as pd
pd.__version__

'1.5.3'

# DB 연결

In [6]:
conn = sqlite3.connect('data/ch15_example.db')
conn

In [10]:
# 커서객체(sql문 실행) 생성
cursor = conn.cursor()
cursor

In [11]:
cursor.execute('''
    CREATE TABLE MEMBER(
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
    )
''')

# insert, update, delet전송

In [13]:
cursor.execute('''
    INSERT INTO MEMBER VALUES('홍길동', 20, 'h@h.com')
''')
print('수행 결과 행수:', cursor.rowcount)

수행 결과 행수: 1


In [21]:
sql = "INSERT INTO MEMBER VALUES ('박길동', 20, 'H@H.COM')"
print(sql)
cursor.execute(sql)
print('수행결과 행수:', cursor.rowcount)

INSERT INTO MEMBER VALUES ('박길동', 20, 'H@H.COM')
수행결과 행수: 1


In [22]:
conn.commit()

### select 전송

In [18]:
cursor.execute('''
    SELECT NAME, AGE+1 NEXTAGE, EMAIL FROM MEMBER
''')

In [24]:
# SELCET 전송
#   select 문 실행결과를 받는 함수
#    fetchone() : 결과를 한행씩 받을 때(투플 홍길동, 21, h@)
#    cursor.fetchall() : 결과를 모두 받을 때 (튜플list)
#    cursor.getchmany(n) : 결과를 n행 받을 때(튜플 list)


# INSERT, UPDATE, DELETE 전송 : cursor.rowcount
print(cursor.fetchall())

[]


In [25]:
cursor.execute('''
    SELECT NAME, AGE+1 NEXTAGE, EMAIL FROM MEMBER order by nextage
''')
members = cursor.fetchmany(3)
members

[('홍길동', 21, 'h@h.com'), ('홍길동', 21, 'h@h.com'), ('홍길동', 21, 'H@H.COM')]

In [26]:
cursor.execute('''
    SELECT NAME, AGE+1 NEXTAGE, EMAIL FROM MEMBER order by nextage
''')
members = cursor.fetchone()
members

('홍길동', 21, 'h@h.com')

In [27]:
cursor.fetchall()

[('홍길동', 21, 'h@h.com'),
 ('홍길동', 21, 'H@H.COM'),
 ('박길동', 21, 'H@H.COM'),
 ('박길동', 21, 'H@H.COM')]

In [31]:
# 데이터 한줄씩 가져오기
cursor.execute('''
    SELECT NAME, AGE+1 NEXTAGE, EMAIL FROM MEMBER order by nextage
''')
members = []
while True:
    member = cursor.fetchone()
    if member is None:
        break
    print(member)
    members.append({'name':member[0], 'age':member[1], 'email':member[2]})
import pandas as pd
pd.DataFrame(members)

('홍길동', 21, 'h@h.com')
('홍길동', 21, 'h@h.com')
('홍길동', 21, 'H@H.COM')
('박길동', 21, 'H@H.COM')
('박길동', 21, 'H@H.COM')


,name,age,email
0,홍길동,21,h@h.com
1,홍길동,21,h@h.com
2,홍길동,21,H@H.COM
3,박길동,21,H@H.COM
4,박길동,21,H@H.COM


In [32]:
cursor.execute('''
    SELECT NAME, AGE+1 NEXTAGE, EMAIL FROM MEMBER order by nextage
''')
members = cursor.fetchall()
df = pd.DataFrame(members)
df

,0,1,2
0,홍길동,21,h@h.com
1,홍길동,21,h@h.com
2,홍길동,21,H@H.COM
3,박길동,21,H@H.COM
4,박길동,21,H@H.COM


In [33]:
# select문을 수행한 필드 정보
cursor.description

(('NAME', None, None, None, None, None, None),
 ('NEXTAGE', None, None, None, None, None, None),
 ('EMAIL', None, None, None, None, None, None))

In [34]:
[description[0] for description in cursor.description]

['NAME', 'NEXTAGE', 'EMAIL']

## 1.3sql 구문에서 파라미터 사용하기
- qmark(db에 따라 불가한 경우 있음)
- named(추천)

In [37]:
conn = sqlite3.connect('data/ch15_example.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN ('홍길동','김길동')")
cursor.fetchall()

[('홍길동', 20, 'h@h.com'), ('홍길동', 20, 'h@h.com'), ('홍길동', 20, 'H@H.COM')]

In [38]:
conn = sqlite3.connect('data/ch15_example.db')
cursor = conn.cursor()
sql = "SELECT * FROM MEMBER WHERE NAME IN (?,?)"
name1 = input('검색할 이름은?')
name2 = input('검색할 다른이름은?')
cursor.execute(sql,(name1, name2))
cursor.fetchall()

검색할 이름은?홍길동
검색할 다른이름은?박길동


[('홍길동', 20, 'h@h.com'),
 ('홍길동', 20, 'h@h.com'),
 ('홍길동', 20, 'H@H.COM'),
 ('박길동', 20, 'H@H.COM'),
 ('박길동', 20, 'H@H.COM')]

In [39]:
# 파라미터 사용하기 : named 방법 이용

sql = "SELECT * FROM MEMBER WHERE NAME IN (:name1,:name2)"
name1 = input('검색할 이름은?')
name2 = input('검색할 다른이름은?')
cursor.execute(sql,{'name1':name1, 'name2':name2})
cursor.fetchall()

검색할 이름은?홍길동
검색할 다른이름은?김길동


[('홍길동', 20, 'h@h.com'), ('홍길동', 20, 'h@h.com'), ('홍길동', 20, 'H@H.COM')]

In [43]:
# named 방법으로 데이터 입력하기
try:
    name = input('입력할 이름은?')
    age = int(input('입력할 나이는?'))
except ValueError:
    print('유요하지 않는 나이를 입력하시면 18세로 초기화')
    age = 18
finally:
    email = input('입력할 메일은?')
cursor.execute('INSERT INTO MEMVER VALUES (:name, :age, : email)',
              {'name':name, 'age':age, 'email':email})
conn.commit()
if cursor.rowcount == 1:
    print('입력성공')
else:
    print('입력실패')

입력할 메일은?sdf44


KeyboardInterrupt: Interrupted by user

In [44]:
cursor.close()

In [46]:
conn.close()

In [47]:
import cx_Oracle
# conn 얻어오는 방법1
conn = cx_Oracle.connect('scott','tiger','localhost:1521/xe')
conn

<cx_Oracle.Connection to scott@localhost:1521/xe>

In [49]:
cursor = conn.cursor()
cursor.execute('SELECT EMPNO NO, ENAME, JOB, MGR, SAL, COMM FROM EMP')
emps = cursor.fetchall()
for emp in emps:
        print(emp)

(7369, 'SMITH', 'CLERK', 7902, 800.0, None)
(7499, 'ALLEN', 'SALESMAN', 7698, 1600.0, 300.0)
(7521, 'WARD', 'SALESMAN', 7698, 1250.0, 500.0)
(7566, 'JONES', 'MANAGER', 7839, 2975.0, None)
(7654, 'MARTIN', 'SALESMAN', 7698, 1250.0, 1400.0)
(7698, 'BLAKE', 'MANAGER', 7839, 2850.0, None)
(7782, 'CLARK', 'MANAGER', 7839, 2450.0, None)
(7788, 'SCOTT', 'ANALYST', 7566, 3000.0, None)
(7839, 'KING', 'PRESIDENT', None, 5000.0, None)
(7844, 'TURNER', 'SALESMAN', 7698, 1500.0, 0.0)
(7876, 'ADAMS', 'CLERK', 7788, 1100.0, None)
(7900, 'JAMES', 'CLERK', 7698, 950.0, None)
(7902, 'FORD', 'ANALYST', 7566, 3000.0, None)
(7934, 'MILLER', 'CLERK', 7782, 1300.0, None)
(9999, 'HONG', None, None, None, None)


In [50]:
pd.DataFrame(emps,
            colums=[des[0] for des in cursor.description])

TypeError: DataFrame.__init__() got an unexpected keyword argument 'colums'